### table - stage
* result_id -> varchar -> uuid -> PK
* result_title -> varchar
* result_url -> varchar
* engine_id -> int ->  FK
* is_ad -> boolean -> int 0 || 1 (0, 1)
* createdAt -> Date (timestamp)
* updatedAt -> Date (op)
* keywords -> varchar



### table - engine
* engine_id - int
* engine_name - varchar


## MY SQL CONNECTOR

In [ ]:
##!pip install mysql-connector-python

In [ ]:
import mysql.connector

"""
Helper function to be able to read and update the database in the cloud
"""

def read_db(query):
  """ Connect to AWS RDS MySQL database server """

  conn = None

  try:

    # Connect to server
    conn = mysql.connector.connect(
      host="data-engineering-db.ckoginbnrheb.us-east-1.rds.amazonaws.com",
      port=3306,
      user="admin",
      password="JO1ZyYOVdylEo6UYAxCD"
    )

    # Get a cursor
    cur = conn.cursor()

    # Execute a query
    cur.execute(query)

    # Fetch the result from the query
    output = cur.fetchall()
    print("Success reading from database!")

    # Close the communication with MySQL
    cur.close()

  except Exception as e:
    print("Error encountered: ", str(e))

  finally:
    if conn is not None:
      conn.close()
      print('Database connection closed.')
  
  return output


def update_db(query):
  """ Connect to AWS RDS MySQL database server """

  conn = None

  try:

    # Connect to server
    conn = mysql.connector.connect(
      host="data-engineering-db.ckoginbnrheb.us-east-1.rds.amazonaws.com",
      port=3306,
      user="admin",
      password="JO1ZyYOVdylEo6UYAxCD"
    )

    # Get a cursor
    cur = conn.cursor()

    # Execute a query
    cur.execute(query)
    print("Success updating database!")

    # Commit the changes to the database
    conn.commit()

    # Close the communication with MySQL
    cur.close()

  except Exception as e:
    print("Error encountered: ", str(e))

  finally:
    if conn is not None:
      conn.close()
      print('Database connection closed.')


In [ ]:
read_db('SELECT CURDATE()')

Success reading from database!
Database connection closed.


[(datetime.date(2022, 3, 21),)]

In [ ]:
"""
SQL Code to create the new tables in the database

"""

db_name = "project_master"
table_1_name = "stage"
table_2_name = "engine"

sql_statement_1=f"""
              CREATE TABLE IF NOT EXISTS {db_name}.{table_2_name}(
                engine_id INT NOT NULL AUTO_INCREMENT,
                engine_name TEXT NOT NULL,
                PRIMARY KEY (engine_id)
              );
              """

sql_statement_2=f"""
              CREATE TABLE IF NOT EXISTS {db_name}.{table_1_name}(
                result_id INT NOT NULL AUTO_INCREMENT,
                result_title TEXT NOT NULL,
                result_url TEXT NOT NULL,
                engine_id INT NOT NULL,
                is_ad BOOLEAN NOT NULL,
                created_at DATETIME NOT NULL,
                query_string TEXT NOT NULL,
                PRIMARY KEY (result_id),
                FOREIGN KEY(engine_id) REFERENCES {db_name}.{table_2_name}(engine_id)
              );
              """

update_db(sql_statement_1)
update_db(sql_statement_2)


Success updating database!
Database connection closed.
Success updating database!
Database connection closed.


In [ ]:
## POPULATE ENGINE TABLE ##

search_engines=['GOOGLE','YAHOO','BING']
sql_header = f"INSERT INTO {db_name}.{table_2_name}(engine_name) VALUES\n"
sql_values=""
for index, item in enumerate(search_engines):
  sql_values += "('"+item+"'),\n"

  if index ==len(search_engines)-1:
    sql_values=sql_values[:-2]+';'

sql_statement = sql_header+sql_values
update_db(sql_statement)

Success updating database!
Database connection closed.


In [ ]:
update_db('DROP TABLE project_master.stage')
update_db('DROP TABLE project_master.engine')

Success updating database!
Database connection closed.
Success updating database!
Database connection closed.


In [ ]:
# google search console 

import requests, sys, webbrowser, bs4

In [ ]:
from bs4 import BeautifulSoup
import requests, json

class SearchEngine:
    def __init__(self, num=5) -> None:
        # self._keywords = "+".join(keywords.split(" ")) # sample: Climate+Change
        self.headers= {
          'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
        }
        self._num = num
        self._google_data = []
        self._yahoo_data = []
        self._bing_data = []

    def keyword_parser(self, keyword):
      return "+".join(keyword.split(" "))


    def google_search(self, keyword):
        #### made headers a class var. delete if okay
        # headers = {
        #   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
        #   }
        html = requests.get('https://www.google.com/search?q='+ self.keyword_parser(keyword) +'&num='+ str(self._num), headers=self.headers)
        soup = BeautifulSoup(html.text, 'lxml')
        linkElements = soup.select_one("#search").select_one("div").find("div")

        # Check if there are complex results
        real_linkElements = linkElements.select(".TzHB6b") if len(linkElements) <= 1 else linkElements

        for link in real_linkElements:
            try:
              title_tags = link.select("h3")
              for title_tag in title_tags:
                  a_tag = title_tag.find_parent("a") if title_tag else None
                  if title_tag != None and a_tag != None:
                      self._google_data.append({
                          'title' :  title_tag.text,
                          'link' : a_tag['href'] 
                      })
            except:
              continue


    def yahoo_search(self, keyword):
      
     for i in range(int(self._num/5)):

      html = requests.get("https://search.yahoo.com/search?p="+self.keyword_parser(keyword) +"&pz="+str(10)+"&b="+str(i*10), headers=self.headers)
      soup = bs4.BeautifulSoup(html.text, "lxml")
      h3s = soup.findAll('h3')



      for item in h3s:
        
        try:  
            sub_soup = bs4.BeautifulSoup(str(item.a).replace(str(item.a.span),''), "lxml")

            if sub_soup.a.text is None:
              continue
            self._yahoo_data.append({
                            'title' : sub_soup.a.text,
                            'link' : sub_soup.a['href']
                        })

        except:
          continue


    def bing_search(self, keyword):
        
        html = requests.get("https://www.bing.com/search?q="+ self.keyword_parser(keyword) +'&count='+ str(self._num), headers=self.headers)
        soup = bs4.BeautifulSoup(html.text, "lxml")
        h2s = soup.findAll('h2')

        for item in h2s:
          title=''
          link=''

          try:
            if 'videos/search' in item.a['href'] or 'news/search' in item.a['href'] or 'images/search' in item.a['href']:
              link = 'https://bing.com'+item.a['href']
            else:
              link = item.a['href']

            title = item.text

            self._bing_data.append({
                        'title' : title,
                        'link' : link
                    })

          except:
            continue

    def full_search(self, keyword):
        self.google_search(keyword);
        self.bing_search(keyword);
        self.yahoo_search(keyword);
        return {"GOOGLE":self._google_data,  "BING":self._bing_data, "YAHOO": self._yahoo_data}

    
    def print_google_search_results(self):
        print(len(self._google_data))
        print(json.dumps(self._google_data, indent=2, ensure_ascii=False))

    def print_yahoo_search_results(self):
        print(len(self._yahoo_data))
        print(json.dumps(self._yahoo_data, indent=2, ensure_ascii=False))

    def print_bing_search_results(self):
        print(len(self._bing_data))
        print(json.dumps(self._bing_data, indent=2, ensure_ascii=False))


In [ ]:
from datetime import datetime

se = SearchEngine(5)
q_string = "test"
results = se.full_search(q_string)

## POPULATE STAGE TABLE ##

sql_header=f"INSERT INTO {db_name}.{table_1_name}(result_title,result_url,engine_id,is_ad,created_at,query_string) VALUES\n"
sql_values=""
for key, value in results.items():

  for val in value:
    sql_values += f"""('{val['title']}','{val['link']}',(SELECT engine_id FROM project_master.engine WHERE engine_name='{key}'),'0','{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}','{q_string}'),\n"""

sql_values=sql_values[:-2]+';'
sql_statement = sql_header+sql_values
# print(sql_statement)

update_db(sql_statement)



Success updating database!
Database connection closed.


In [ ]:
# test cases: Climate Change, Covid-19, covid symptoms, ccny, ccny academic calendar 
# test = SearchEngine("test",50)
# test.google_search()
# test.print_google_search_results()

# test.bing_search()
# test.print_bing_search_results()

# test.yahoo_search()
# test.print_yahoo_search_results()

71
[
  {
    "title": "Test Definition & Meaning - Merriam-Webster",
    "link": "https://r.search.yahoo.com/_ylt=A0geJaIxzzdiqrAAsSJXNyoA;_ylu=Y29sbwNiZjEEcG9zAzEEdnRpZAMEc2VjA3Ny/RV=2/RE=1647853490/RO=10/RU=https%3a%2f%2fwww.merriam-webster.com%2fdictionary%2ftest/RK=2/RS=5tz0r7Hc2GPwFVSHkW0zNMgzToI-"
  },
  {
    "title": "Top Stories",
    "link": "https://news.search.yahoo.com/search;_ylt=A0geJaIxzzdiqrAAsyJXNyoA;_ylu=Y29sbwNiZjEEcG9zAzIEdnRpZAMEc2VjA3Nj?p=test&fr2=p%3As%2Cv%3Aw%2Cm%3Anewsdd_sna_t%2Cct%3Avm-vmonly"
  },
  {
    "title": "Test - definition of test by The Free Dictionary",
    "link": "https://r.search.yahoo.com/_ylt=A0geJaIxzzdiqrAAySJXNyoA;_ylu=Y29sbwNiZjEEcG9zAzIEdnRpZAMEc2VjA3Ny/RV=2/RE=1647853490/RO=10/RU=https%3a%2f%2fwww.thefreedictionary.com%2ftest/RK=2/RS=ie7LQvpvw4gHA284T4AF1eI.e5Y-"
  },
  {
    "title": "Images",
    "link": "https://images.search.yahoo.com/search/images;_ylt=A0geJaIxzzdiqrAAzSJXNyoA;_ylu=Y29sbwNiZjEEcG9zAzQEdnRpZAMEc2VjA3Nj?p=test"
  },

## Resources
* [Python MySQL](https://www.w3schools.com/python/python_mysql_getstarted.asp)
* [Prettify Your Data Structures With Pretty Print in Python](https://realpython.com/python-pretty-print/)
* [Selenium with Python](https://selenium-python.readthedocs.io/)
* [(NOT FREE) SerpAPI - Scrape Google and other search engines from our fast, easy, and complete API](https://serpapi.com/)
* [BeautifulSoup tutorial: Scraping web pages with Python](https://www.scrapingbee.com/blog/python-web-scraping-beautiful-soup/)
* [Google Search Console](https://search.google.com/search-console/about)

# Niloufar

In [ ]:
# http://httpbin.org/get

In [ ]:
# import numpy as np
# import requests, sys, webbrowser, bs4


# def google_search(keyword):
#   headers = {'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36'}

#   res = requests.get("https://google.com/search?q="+str(keyword))
#   res.raise_for_status()
#   soup = bs4.BeautifulSoup(res.text, "html.parser")
#   print(soup.prettify())
 
#   #linkElements=soup.find_all( 'a' )
#   linkElements = soup.find_all( "div" , class_='BNeawe')
#   for info in linkElements:
#     print(info.getText())
#     print("------")


In [ ]:
# Edited by Jiawei

import requests, sys, webbrowser, bs4
headers = {
  "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36"
  }
for page in range(2):
  res = requests.get("https://google.com/search?q="+ str("Climate Change") + str(page) + '/',headers=headers)
  # res = requests.get("https://google.com/search?q="+ str("Climate Change"),headers=headers)
  res.raise_for_status()
  soup = bs4.BeautifulSoup(res.text, "html.parser")
  linkElements = soup.select('.kCrYT > a')
  for i in linkElements:
    # print(i.get('href'))
    print(i)

In [ ]:
import requests, sys, webbrowser, bs4
headers = {
  "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36"
  }
for page in range(2):
  res = requests.get("https://google.com/search?q="+ str("Climate Change") + str(page) + '/',headers=headers)
  res.raise_for_status()
  soup = bs4.BeautifulSoup(res.text, "html.parser")
  linkElements = soup.select('.kCrYT > a')
  for i in linkElements[0:500]:
    print(i.get('href'))
    print(i)
    
  #for title in soup.find_all( "div" , class_='BNeawe'):
    #print("Title of the website is : ")
    #print(title.get_text())
    




In [ ]:
 #linkElements=soup.find_all( 'a' )
linkElements = soup.find_all( "div" , class_='BNeawe')
#print(linkElements)

# for info in linkElements:
#   print(info.getText())
#   print("------")

# JUAN

## BING

In [ ]:
import requests, sys, webbrowser, bs4
import numpy as np
import json
def bing_search(keyword, num=50):
  headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
  }
  html = requests.get("https://www.bing.com/search?q="+str(keyword)+"&count="+str(num), headers=headers)
  soup = bs4.BeautifulSoup(html.text, "lxml")
  h2s = soup.findAll('h2')

  print(len(h2s))

  for item in h2s:

    try:
      if 'videos/search' in item.a['href']:
        print('href','https://bing.com'+item.a['href'])
      else:
        print('href',item.a['href'])
      print(item.text)

    except:
      continue

bing_search('CCNY')

# <a h="ID=SERP,5166.1" href="https://www.ccny.cuny.edu/">
#          The City College of New York
# </a>

# <a h="ID=SERP,5209.1" href="https://en.wikipedia.org/wiki/City_College_of_New_York">
#          City College of New York - Wikipedia
#         </a>
  

49
href https://www.ccny.cuny.edu/
The City College of New York
href https://en.wikipedia.org/wiki/City_College_of_New_York
City College of New York - Wikipedia
href https://ccny.edu/
CCNY - City Central
href https://ccny.com/
CCNY - Formally Computer Connection of New York
href https://bing.com/videos/search?q=ccny&qpvt=CCNY&FORM=VDRE
Videos of CCNY
href https://giving.ccny.cuny.edu/
Home - City College of New York
href https://ccnyinc.org/
CCNY | The Human Side of Analytics
href https://childcenterny.org/
Child Center of NY
href https://www.prepsportswear.com/college/us/new-york/new-york/city-college-of-new-york-beavers?schoolid=2063560
City College of New York Beavers Apparel Store | Prep ...
href https://portal.ccny.cuny.edu/depts/hr/timesheet_hr/signin.php
TimeSheet For College Assistants - City University of New York
href https://ssa.ccny.cuny.edu/
The Bernard and Anne Spitzer School of Architecture
href https://ccnyathletics.com/
The City College of New York Athletics - Official

Huihuang Liu


## YAHOO

In [ ]:
from typing import TYPE_CHECKING
import requests, sys, webbrowser, bs4
import numpy as np
import json

_yahoo_data=[]
def yahoo_search(keyword, num=50):
  headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
  }

  for i in range(int(num/10)):

    html = requests.get("https://search.yahoo.com/search?q=test"+"&pz="+str(10)+"&b="+str(i*10), headers=headers)
    soup = bs4.BeautifulSoup(html.text, "lxml")
    h3s = soup.findAll('h3')



    for item in h3s:
      # print(item)
      # <class 'bs4.element.Tag'>
      
      try:  
        # if item is not None:

          sub_soup = bs4.BeautifulSoup(str(item.a).replace(str(item.a.span),''), "lxml")

          if sub_soup.a.text is None:
            continue

          # print(new_soup.a)

          # print(item.a)
          print('Title:   ',sub_soup.a.text)
          print('Link:   ',sub_soup.a['href'])
          _yahoo_data.append({
                          'title' : item.text,
                          'link' : item.a['href']
                      })
          # print('')
        # else:
        #   print('href',item.a['href'])
        # print(item.text)

      except:
        continue


yahoo_search('CCNY')
print(len(_yahoo_data))


Title:    Test Definition & Meaning - Merriam-Webster
Link:    https://r.search.yahoo.com/_ylt=AwrJ61cDzjdigpYA5FtXNyoA;_ylu=Y29sbwNiZjEEcG9zAzEEdnRpZAMEc2VjA3Ny/RV=2/RE=1647853187/RO=10/RU=https%3a%2f%2fwww.merriam-webster.com%2fdictionary%2ftest/RK=2/RS=RyPFE9SIFQewjvfJNbI88kkf0lo-
Title:    Top Stories
Link:    https://news.search.yahoo.com/search;_ylt=AwrJ61cDzjdigpYA5ltXNyoA;_ylu=Y29sbwNiZjEEcG9zAzIEdnRpZAMEc2VjA3Nj?p=test&fr2=p%3As%2Cv%3Aw%2Cm%3Anewsdd_sna_t%2Cct%3Avm-vmonly
Title:    Test - definition of test by The Free Dictionary
Link:    https://r.search.yahoo.com/_ylt=AwrJ61cDzjdigpYA_FtXNyoA;_ylu=Y29sbwNiZjEEcG9zAzIEdnRpZAMEc2VjA3Ny/RV=2/RE=1647853187/RO=10/RU=https%3a%2f%2fwww.thefreedictionary.com%2ftest/RK=2/RS=.obItWvI.86G3CslTDA0jTAMvz4-
Title:    Images
Link:    https://images.search.yahoo.com/search/images;_ylt=AwrJ61cDzjdigpYAAFxXNyoA;_ylu=Y29sbwNiZjEEcG9zAzQEdnRpZAMEc2VjA3Nj?p=test
Title:    80 Synonyms of TEST - Merriam-Webster
Link:    https://r.search.yahoo.com/

In [ ]:
import requests, sys, webbrowser, bs4
import numpy as np
import json
def yahoo_search(keyword, num=50):
  headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36',
  }

  html = requests.get("https://search.yahoo.com/search?q="+str(keyword)+"&n="+str(num))
  soup = bs4.BeautifulSoup(html.text, "lxml")
  h3s = soup.findAll('h3')
  #print(h3s)  
  for item in h3s:
    
    try:  
      if 'a' in item.a['href']:
        print('Title:   ',item.text)
        print('Link:   ',item.a['href'])
        print('')
#       else:
#         print('href',item.a['href'])
#       print(item.text)

    except:
      continue

print(yahoo_search('CCNY'))

yahoo = "https://search.yahoo.com/search?q=" + query + "&n=" + str(no)
#yahoo = "https://search.yahoo.com/search?p=" + query
raw_page = requests.get(yahoo) 
results = re.findall(r'(?<=<h3 class="r"><a href="/url\?q=).*?(?=&amp)', str(raw_page))
res = list(set(results))[0:30] #Provides 30 unique of the 35 we requested above

print(res)

Title:    Images
Link:    https://images.search.yahoo.com/search/images?p=CCNY

Title:    en.wikipedia.org › wiki › City_College_of_New_YorkCity College of New York - Wikipedia
Link:    https://en.wikipedia.org/wiki/City_College_of_New_York

Title:    What does CCNY stand for? - Abbreviations.com
Link:    https://www.acronymfinder.com/City-College-of-New-York-(City-University-of-New-York)-(CCNY).html

Title:    Does The City College of New York (CCNY) have a good ...
Link:    https://www.quora.com/Is-CUNY-Baruch-good-for-computer-science

Title:    Computer Technician Salary - ComputerCareers.org
Link:    https://www.computercareers.org/computer-technician-salary/

Title:    What is a CCNY House Plan
Link:    https://www.ccny.cuny.edu/giving/ccny-valentines-day

Title:    ccnyathletics.comThe City College of New York Athletics - Official Athletics ...
Link:    https://ccnyathletics.com/

None


NameError: ignored

In [ ]:
import requests, sys, webbrowser, bs4
import numpy as np
import json
links=[]
def yahoo_search(keyword):
  headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
  }
  
  html = requests.get("https://search.yahoo.com/search?p="+str(keyword), headers=headers)
  soup = bs4.BeautifulSoup(html.text, "html.parser")
  #print(soup.prettify())
  url_link = soup.find_all("span", {"class":"d-ib p-abs t-0 l-0 fz-14 lh-20 fc-obsidian wr-bw ls-n pb-4"})
  title = soup.find_all("span", {"title"})
  #print(url_link)
  for i in url_link:
    print(i.get_text())
  for j in title:
    print(j.get_text())




